In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from os import listdir
import cv2
import dlib
from imutils import face_utils
import argparse
from google.colab.patches import cv2_imshow
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
# import face_recognition

#mtcnn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_base = '/content/drive/My Drive/Colab Notebooks/ml_project/ForRating/'
# path = './ForRating/'
files = [path_base+f for f in listdir(path_base)]


In [4]:
file_name = '/content/drive/My Drive/Colab Notebooks/ml_project/FER_Ground_Truth/ActionUnitExtraction.csv'
# file_name = '/content/drive/My Drive/Colab Notebooks/ml_project/FER_Ground_Truth/Landmarks2.csv'

# path = './ForRating/'
# files = [path_base+f for f in listdir(path_base)]


## action unit clustering 

In [8]:
df = pd.read_csv(file_name)
df.head()

,frame,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,filename,labels_AMY
0,1,0.98,1,0.70,0.0,0.28,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.29,0.47,0.00,0.17,0.00,1.30,0.97,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,19-79,anger
1,2,0.88,1,0.42,0.0,0.44,0.11,0.00,0.11,0.0,0.00,0.00,0.51,0.10,0.63,0.12,0.80,0.34,2.06,0.32,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,19-126,anger
2,3,0.98,1,0.61,0.0,0.47,0.11,0.20,0.11,0.0,0.03,0.05,1.16,0.01,0.47,0.12,0.75,0.54,1.80,0.46,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,19-187,anger
3,4,0.98,1,0.81,0.0,0.36,0.11,0.37,0.00,0.0,0.06,0.14,1.45,0.01,0.12,0.00,0.37,0.40,0.54,0.79,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,19-203,sadness
4,5,0.98,1,1.18,0.0,0.41,0.00,0.40,0.00,0.0,0.06,0.22,1.81,0.03,0.08,0.00,0.00,0.40,0.25,1.45,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,19-260,sadness


In [6]:
df['Labels'].unique()

KeyError: ignored

In [9]:
def group_label(df):
  if (df['Labels'] == "['anger']") or (df['Labels'] == "['sadness']")\
   or (df['Labels'] == "['fear']")  or (df['Labels'] == "['contempt']")\
    or (df['Labels'] == "['disgust']") :
    return 'negative'
  elif (df['Labels'] == "['happiness']") or (df['Labels'] == "['surprise']"):
    return 'positive'
  else:
    return 'neutral'


df = df.assign(label_highlevel = df.apply(group_label, axis=1))

KeyError: ignored

In [ ]:
df

,Unnamed: 0,Landmarks,Labels,label_highlevel
0,CIRVR19 (126).jpg,"[[-5, 63], [-2, 95], [4, 125], [14, 153], [29,...",['anger'],negative
1,CIRVR19 (1374).jpg,"[[0, 74], [1, 108], [4, 139], [13, 167], [30, ...",['anger'],negative
2,CIRVR19 (1402).jpg,"[[-6, 87], [-4, 120], [-1, 152], [7, 183], [24...",['anger'],negative
3,CIRVR19 (1403).jpg,"[[-4, 89], [-3, 123], [-1, 156], [5, 187], [20...",['anger'],negative
4,CIRVR19 (1535).jpg,"[[-1, 98], [4, 128], [12, 155], [20, 181], [34...",['anger'],negative
...,...,...,...,...
161,CIRVR21 (1133).jpg,"[[1, 110], [4, 139], [10, 170], [17, 198], [31...",['surprise'],positive
162,CIRVR21 (1134).jpg,"[[1, 108], [4, 138], [10, 169], [19, 197], [32...",['surprise'],positive
163,CIRVR21 (3571).jpg,"[[-1, 95], [-1, 120], [-1, 146], [3, 171], [14...",['surprise'],positive
164,CIRVR21 (713).jpg,"[[0, 117], [2, 146], [7, 175], [15, 203], [29,...",['surprise'],positive


In [10]:
def group_label(df):
  if (df['labels_AMY'] == 'anger') or (df['labels_AMY'] == 'sadness')\
   or (df['labels_AMY'] == 'fear')  or (df['labels_AMY'] == 'contempt')\
    or (df['labels_AMY'] == 'disgust') or (df['labels_AMY'] == 'disgust '):
    return 'negative'
  elif (df['labels_AMY'] == 'happiness') or (df['labels_AMY'] == 'surprise'):
    return 'positive'
  else:
    return 'neutral'


df = df.assign(label_highlevel = df.apply(group_label, axis=1))

In [11]:
df.head()

,frame,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,filename,labels_AMY,label_highlevel
0,1,0.98,1,0.70,0.0,0.28,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.29,0.47,0.00,0.17,0.00,1.30,0.97,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,19-79,anger,negative
1,2,0.88,1,0.42,0.0,0.44,0.11,0.00,0.11,0.0,0.00,0.00,0.51,0.10,0.63,0.12,0.80,0.34,2.06,0.32,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,19-126,anger,negative
2,3,0.98,1,0.61,0.0,0.47,0.11,0.20,0.11,0.0,0.03,0.05,1.16,0.01,0.47,0.12,0.75,0.54,1.80,0.46,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,19-187,anger,negative
3,4,0.98,1,0.81,0.0,0.36,0.11,0.37,0.00,0.0,0.06,0.14,1.45,0.01,0.12,0.00,0.37,0.40,0.54,0.79,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,19-203,sadness,negative
4,5,0.98,1,1.18,0.0,0.41,0.00,0.40,0.00,0.0,0.06,0.22,1.81,0.03,0.08,0.00,0.00,0.40,0.25,1.45,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,19-260,sadness,negative


In [12]:
data = df.iloc[:,0:-3]

In [13]:
data = df['Landmarks']


KeyError: ignored

In [ ]:
import json
data_arr = []
for entry in data:
  tmp =[]
  arr = json.loads(entry)
  for en in arr:
    tmp.extend(en)
  data_arr.append(tmp)

In [ ]:
data_arr = np.array(data_arr)
np.shape(data_arr)

(166, 136)

In [ ]:
kmeans = KMeans(n_clusters=3).fit(data_arr)
df['kmeans'] = kmeans.labels_

In [ ]:
spectralClustering = SpectralClustering(n_clusters=3,  random_state=0).fit(data_arr)
df['spectral'] = spectralClustering.labels_

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."


In [14]:
kmeans = KMeans(n_clusters=3).fit(data)
df['kmeans'] = kmeans.labels_

In [ ]:
spectralClustering = SpectralClustering(n_clusters=3,  random_state=0).fit(data)
df['spectral'] = spectralClustering.labels_

In [ ]:
print('# positive:{}'.format(df['label_highlevel'].value_counts()['positive']))
print('# negative:{}'.format(df['label_highlevel'].value_counts()['negative']))
print('# neutral:{}'.format(df['label_highlevel'].value_counts()['neutral']))

# positive:45
# negative:93
# neutral:62


In [ ]:
print('kmeans method')
print('# 0:{}'.format(df['kmeans'].value_counts()[0]))
print('# 1:{}'.format(df['kmeans'].value_counts()[1]))
print('# 2:{}'.format(df['kmeans'].value_counts()[2]))

kmeans method
# 0:32
# 1:79
# 2:89


In [ ]:
print('spectral clustering method')
print('# 0:{}'.format(df['spectral'].value_counts()[0]))
print('# 1:{}'.format(df['spectral'].value_counts()[1]))
print('# 2:{}'.format(df['spectral'].value_counts()[2]))


spectral clustering method
# 0:177
# 1:9
# 2:14


In [ ]:
print('kmeans 0')
print(df.loc[df['kmeans']==0]['label_highlevel'].value_counts())
print('kmeans 1')
print(df.loc[df['kmeans']==1]['label_highlevel'].value_counts())
print('kmeans 2')
print(df.loc[df['kmeans']==2]['label_highlevel'].value_counts())


kmeans 0
positive    13
neutral     12
negative     7
Name: label_highlevel, dtype: int64
kmeans 1
negative    40
neutral     29
positive    10
Name: label_highlevel, dtype: int64
kmeans 2
negative    46
positive    22
neutral     21
Name: label_highlevel, dtype: int64


In [ ]:
print('spectral 0')
print(df.loc[df['spectral']==0]['label_highlevel'].value_counts())
print('spectral 1')
print(df.loc[df['spectral']==1]['label_highlevel'].value_counts())
print('spectral 2')
print(df.loc[df['spectral']==2]['label_highlevel'].value_counts())


spectral 0
negative    92
neutral     52
positive    33
Name: label_highlevel, dtype: int64
spectral 1
neutral    9
Name: label_highlevel, dtype: int64
spectral 2
positive    12
negative     1
neutral      1
Name: label_highlevel, dtype: int64


In [16]:
y_pos = (df['label_highlevel'] == 'positive').astype(int).tolist()
y_neu = (df['label_highlevel'] == 'neutral').astype(int).tolist()
y_neg = (df['label_highlevel'] == 'negative').astype(int).tolist()


In [23]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_processed = data.copy()
y_processed = y_pos.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
print(accuracy_score(y_test, preds))

0.72


In [20]:
import xgboost as xgb #tree boosting algorithm / lightGBM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_processed = data_arr.copy()
y_processed = y_pos.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
print(accuracy_score(y_test, preds))

NameError: ignored

In [ ]:
import lightgbm as lgb. #lightgbm alrotihm on positive group


model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')
# Predict
preds = model.predict(X_test)

# Score
print(accuracy_score(y_test, preds))

[20]	training's binary_logloss: 0.43027	training's binary_logloss: 0.43027	valid_0's binary_logloss: 0.688033	valid_0's binary_logloss: 0.688033
[40]	training's binary_logloss: 0.312229	training's binary_logloss: 0.312229	valid_0's binary_logloss: 0.679214	valid_0's binary_logloss: 0.679214
[60]	training's binary_logloss: 0.23758	training's binary_logloss: 0.23758	valid_0's binary_logloss: 0.704823	valid_0's binary_logloss: 0.704823
[80]	training's binary_logloss: 0.183417	training's binary_logloss: 0.183417	valid_0's binary_logloss: 0.736436	valid_0's binary_logloss: 0.736436
[100]	training's binary_logloss: 0.146062	training's binary_logloss: 0.146062	valid_0's binary_logloss: 0.777246	valid_0's binary_logloss: 0.777246
0.6428571428571429


In [24]:
X_processed = data.copy()
y_processed = y_neu.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
print(accuracy_score(y_test, preds))

0.62


In [25]:
X_processed = data.copy()
y_processed = y_neg.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
print(accuracy_score(y_test, preds))

0.66


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}
# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
_ = grid_cv.fit(X_processed, y_processed)


In [ ]:
grid_cv.best_score_

0.42309267793138766

In [ ]:
grid_cv.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.25,
 'learning_rate': 0.01,
 'max_depth': 3,
 'reg_lambda': 0,
 'scale_pos_weight': 5,
 'subsample': 0.8}